In [1]:
# step10 create ENCSR618WVK_ClusterMetadata (that has RNA available)
# “{DatasetID}_CellClusterAssignment.tsv”

# bc				CellClusterID	RNAUMI	RNAGenes	ATACFragments	[additional columns ok]
# ATCGATCGATCGATCG_Dataset1Lane1	Xu2020_Cluster1	352	250	525		[e.g. % ribo, % mito]


# qc:
# total_counts	duplicate_counts	umis

In [2]:
%run tsv_files_utils_from_analysis.ipynb

In [3]:
# The only file with RNA

In [4]:
# tagAlign_clusterMetadata_tsv_list = ["./clusters/ENCSR618WVK/ENCSR618WVK_ClusterMetadata.tsv"]
# tagAlign_clusterMetadata_tsv_list

In [5]:
tagAlign_clusterMetadata_tsv_list = !ls ./clusters/*/*_ClusterMetadata.tsv
tagAlign_clusterMetadata_tsv_list = list(tagAlign_clusterMetadata_tsv_list)
tagAlign_clusterMetadata_tsv_list[0:2]

['./clusters/ENCSR023FME/ENCSR023FME_ClusterMetadata.tsv',
 './clusters/ENCSR052DKH/ENCSR052DKH_ClusterMetadata.tsv']

In [6]:
# tagAlign_clusterMetadata_tsv_list = ["./clusters/ENCSR618WVK/ENCSR618WVK_ClusterMetadata.tsv"]
# tagAlign_clusterMetadata_tsv_list

In [7]:
ENCSR618WVK_RNA_QC_file = "/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/filtered_h5_count_matrix/ENCSR618WVK.qc.rna.hg38.barcode.metadata.tsv"
# ENCSR618WVK_barcodes_atacFragmetns = "/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR618WVK/ENCSR618WVK_barcodes_atac_fragments.tsv"




In [8]:
# tagAlign_clusterMetadata_tsv.split("/")[-2]
# dataset_barcodes_atacFragmetns = os.path.join(os.path.dirname(tagAlign_clusterMetadata_tsv),"{}_barcodes_atac_fragments.tsv".format(datasetId))

# dataset_barcodes_atacFragmetns=='./clusters/ENCSR618WVK/ENCSR618WVK_barcodes_atac_fragments.tsv'

In [9]:
tagAlign_clusterMetadata_tsv_list[0:1]

['./clusters/ENCSR023FME/ENCSR023FME_ClusterMetadata.tsv']

In [10]:
# ENCSR618WVK_barcodes_atacFragmetns
# df_umis_genes.head(2)

In [11]:
import os
import pandas as pd
# from the filtered H5
for tagAlign_clusterMetadata_tsv in tagAlign_clusterMetadata_tsv_list:
#debug
# for tagAlign_clusterMetadata_tsv in tagAlign_clusterMetadata_tsv_list[1:2]:
    df_umis_genes= None
    df_barcodes_atac = None
    datasetId = tagAlign_clusterMetadata_tsv.split("/")[-2]
    print("datasetId is ",datasetId)

    # output file
    final_output_file = os.path.join(os.path.dirname(tagAlign_clusterMetadata_tsv),'{}_CellClusterAssignment.tsv'.format(datasetId))
    print("final_output_file is ",final_output_file)
    if os.path.exists(final_output_file) == True:
        print("final_output_file was created already. Nothing to do for ",final_output_file)
        continue
        
    if "ENCSR618WVK" in tagAlign_clusterMetadata_tsv: # has RNA
        df_umis_genes = read_tsv_file_no_index(ENCSR618WVK_RNA_QC_file)
        print(df_umis_genes.head(2))

    # ENCSR618WVK_barcodes_atacFragmetns = "/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR618WVK/ENCSR618WVK_barcodes_atac_fragments.tsv"
    dataset_barcodes_atacFragmetns = os.path.join(os.path.dirname(tagAlign_clusterMetadata_tsv),"{}_barcodes_atac_fragments.tsv".format(datasetId))
    print("dataset_barcodes_atacFragmetns is ",dataset_barcodes_atacFragmetns)
    df_barcodes_atac = read_tsv_file_no_index(dataset_barcodes_atacFragmetns)
    print(df_barcodes_atac.head(2))
    
    df = read_tsv_file_no_index(tagAlign_clusterMetadata_tsv)
    print(df.head(2))
    # df['tagAlignFile']
    # Extracting values from the 'tagAlignFile' column
    tagAlignFile_values = df['tagAlignFile']
    
    # Convert the values to a list (optional)
    tagAlignFile_list = tagAlignFile_values.tolist()
    print("tagAlignFile_list is ",tagAlignFile_list)
    # Create an empty DataFrame to store the combined data
    # combined_df = pd.DataFrame(columns=['bc', 'CellClusterID','RNAUMI','RNAGenes','ATACFragments'])
    # print("combined_df is ",combined_df)
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    # Loop through the files
    for f in tagAlignFile_list:
    # for f in tagAlignFile_list[0:1]: #debug
        tagAlignFile_fullPath = os.path.join(os.path.dirname(tagAlign_clusterMetadata_tsv), f)
        print(f"tagAlignFile_fullPath: {tagAlignFile_fullPath}")
        df_tagAlign = read_tsv_file(tagAlignFile_fullPath)
        print("df_tagAlign.head(2) is ",df_tagAlign)
        # Extract the values from the third column
        barcodes_values = df_tagAlign.iloc[:, 3]
        barcodes_values_list = barcodes_values.tolist()
        print(barcodes_values_list[0:2])
        
        # Create a new DataFrame with the extracted data
        data = {
            # 'CellClusterID': [f] * len(barcodes_values_list),
            'bc': barcodes_values_list
        }
        new_df = pd.DataFrame(data)

        dfs.append(new_df)
    # Concatenate the list of DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)
    print(combined_df.head(2))
    
    result_df = None
    if "ENCSR618WVK" in tagAlign_clusterMetadata_tsv:
        print(df_umis_genes.head(2))
        num_of_unique_barcodes = len(list(set(df_umis_genes['barcode'].tolist())))
        print("num_of_unique_barcodes is ",num_of_unique_barcodes)

        num_of_unique_bc = len(list(set(combined_df['bc'].tolist())))
        print("num_of_unique_bc is ",num_of_unique_bc)

        # Merge DataFrames based on 'barcode' and 'bc' columns
        result_df = pd.merge(df_umis_genes, combined_df, left_on='barcode', right_on='bc', how='inner')
        print(result_df.head(2))

        # Drop duplicated rows based on the 'bc' column
        result_df.drop_duplicates(subset='bc', inplace=True)
        # Drop the 'barcode' column
        result_df.drop('barcode', axis=1, inplace=True)
        print(result_df.head(2))
        # Rename 'umis' and 'genes' columns
        result_df.rename(columns={'umis': 'RNAUMI', 'genes': 'RNAGenes'}, inplace=True)
    else:
        result_df = combined_df
        # Define additional column headers
        additional_columns = ['RNAUMI', 'RNAGenes']

        # Add the additional columns to the DataFrame with no values to make it uniform with other
        # samples that have RNA and ATAC data available
        for col in additional_columns:
            df[col] = None  # Fill with None, NaN, or any other appropriate placeholder
        
        
    result_df_with_atacFragments = pd.merge(result_df, df_barcodes_atac, on=['bc'], how='inner')
    print("\n\n\nresult_df_with_atacFragments head is ",result_df_with_atacFragments.head(2))

    # Drop duplicated rows based on the 'bc' column
    print("before")
    print(len(result_df_with_atacFragments['bc'].to_list()))
    result_df_with_atacFragments.drop_duplicates(subset='bc', inplace=True)
    print("before")
    print(len(result_df_with_atacFragments['bc'].to_list()))
    
    # Reorder columns with 'bc' and 'CellClusterID' as the first columns
    result_df_with_atacFragments = result_df_with_atacFragments[['bc', 'CellClusterID'] + [col for col in result_df_with_atacFragments.columns if col not in ['bc', 'CellClusterID']]]
    print(result_df_with_atacFragments.head(2))
    
    
    
    # Write the combined DataFrame to a TSV file
    result_df_with_atacFragments.to_csv(final_output_file, sep='\t', index=False)


datasetId is  ENCSR023FME
final_output_file is  ./clusters/ENCSR023FME/ENCSR023FME_CellClusterAssignment.tsv
dataset_barcodes_atacFragmetns is  ./clusters/ENCSR023FME/ENCSR023FME_barcodes_atac_fragments.tsv
                                   bc         CellClusterID  ATACFragments
0  ENCSR023FME_AAACGCAAGCAAGCGCCATCGA  ENCSR023FME_Cluster1          11300
1  ENCSR023FME_AAACGCAAGCAAGGGCAAATCA  ENCSR023FME_Cluster1           4926
                                       tagAlignFile         CellClusterID  \
0  tagAlign_ENCSR023FME_Type_I_skeletal_myocyte.tsv  ENCSR023FME_Cluster1   
1               tagAlign_ENCSR023FME_Fibroblast.tsv  ENCSR023FME_Cluster2   

     ManualAnnotationLabel              TissueName  nCells MeanRNAUMIsPerCell  \
0  Type_I_skeletal_myocyte  gastrocnemius medialis    3528                 ()   
1               Fibroblast  gastrocnemius medialis     618                 ()   

   MeanATACFragmentsPerCell  
0                      4172  
1                      2623  
ta

FileNotFoundError: [Errno 2] No such file or directory: './clusters/backup_ENCSR618WVK copy/backup_ENCSR618WVK copy_barcodes_atac_fragments.tsv'

In [ ]:
result_df_with_atacFragments.head(2)

In [ ]:
result_df_with_atacFragments.tail(2)